In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def export(results):
    df=pd.DataFrame(results)
    df.to_csv('dataset.csv',index=False,mode='a',header=False)
    

In [13]:
def extract_rating(rating_a):
    if rating_a is None or rating_a.find('span', class_="main-2") is None:
        return "None"
    else:
        return rating_a.find('span', class_="main-2").text
        
def generate_url(index):
    if index == 1:
        return "https://www.naukri.com/software-engineer-software-development-jobs-in-india?k=software%20engineer%2C%20software%20development&l=india&experience=0&qproductJobSource=2&naukriCampus=true&experience=0&nignbevent_src=jobsearchDeskGNB"
    else:
        return format("https://www.naukri.com/software-engineer-software-development-jobs-in-india-{}".format(index))
    return url        
        
def main(page_jobs):
    info=[]
    for job in page_jobs:
        row1=job.find('div',class_='row1')
        row2 = job.find('div', class_="row2")
        row3 = job.find('div', class_="row3")
        row4 = job.find('div', class_="row4")
        row5 = job.find('div', class_="row5")
        row6 = job.find('div', class_="row6")
        #title
        title=row1.a.text.strip()
        
        #company name
        company_name = row2.span.a.text
        
        #rating
        rating_a = row2.span
        rating = extract_rating(rating_a)
       
        #job details
        job_details = row3.find('div', class_='job-details')
        if job_details and job_details.find('span', class_="exp-wrap"):
            ex_wrap = job_details.find('span', class_="exp-wrap").span.text.strip()
        else:
            ex_wrap = "Not Specified"
       
        #salary
        salary=job_details.find('span', class_="sal-wrap ver-line").span.text.strip()
        
        
        #location
        location=job_details.find('span', class_="loc-wrap ver-line").span.text.strip()
        
        
        #job overview
        job_overview = row4.span.text.strip() if row4 and row4.span else "Not Available"
        

        #skills
        skills_list = []
        if row5 and row5.ul:
            for tech_stack in row5.ul.find_all('li', class_="dot-gt tag-li"):
                skills_list.append(tech_stack.text.strip())
        skills = ', '.join(skills_list) if skills_list else "No skills mentioned"
        
        
        
       #date posted
        date_posted=row6.span.text.strip()
        
        
        #URL
        url=row1.find('a').get('href')
        data={
            'Title':title,
            'Company Name':company_name,
            'Rating':rating,
            'Experience':ex_wrap,
            'Salary':salary,
            'Location':location,
            'Job Overview':job_overview,
            'Date Posted':date_posted,
            'Skills':skills,
            'URL':url
        }

        info.append(data)
    export(info)
        
options = webdriver.ChromeOptions() 
options.headless = True 
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

start_page = 1
page_end = 51
for i in range(start_page, page_end):
    print(i)
    url = generate_url(i)
    driver.get(url)
    # sleep for 5-10 seconds randomly just to let it load
    sleep(randint(5, 10))
    get_url = driver.current_url
    if get_url == url:
        page_source = driver.page_source

    # Generate the soup
    soup = BeautifulSoup(page_source, 'html.parser')
    page_soup = soup.find_all("div", class_="srp-jobtuple-wrapper")
    main(page_soup)
  


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
